In [1]:
from google.colab import files
uploaded = files.upload()

Saving test_dataset0207.zip to test_dataset0207.zip


In [2]:
!mkdir datasets

In [ ]:
!unzip test_dataset0207.zip

In [4]:
!mv test_dataset/ datasets/

# Formal

In [5]:
import yaml
import os
import yaml
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

In [6]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # size: 512x512
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # size: 256x256
            nn.Conv2d(16, 32, kernel_size=3, padding=1),  # size: 256x256
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # size: 128x128
            nn.Conv2d(32, 32, kernel_size=3, padding=1),  # size: 128x128
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # size: 64x64
            nn.Flatten(),
            nn.Linear(32 * 64 * 64, 1024),  # size: 1024
        )

    def forward(self, x):
        x = self.features(x)
        return x

In [7]:
class Decoder(nn.Module):
    def __init__(self, input_size, output_size):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [8]:
class MultiTailDecoder(nn.Module):
    def __init__(self, input_size, classification_sizes=None, regression_size=None):
        super(MultiTailDecoder, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        self.classification_tails = [nn.Linear(128, size) for size in classification_sizes] if classification_sizes else []
        self.regression_tail = nn.Linear(128, regression_size) if regression_size else None

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        classification_outputs = [tail(x) for tail in self.classification_tails] if self.classification_tails else []
        regression_output = self.regression_tail(x) if self.regression_tail else None
        return classification_outputs, regression_output

In [9]:
class ParamAwareMultiTailDecoder(nn.Module):
    def __init__(self, input_size, classification_params=None, regression_params=None):
        super(ParamAwareMultiTailDecoder, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        # {param_name: tail}
        self.classification_tails = nn.ModuleDict(
                {
                    param_name: nn.Sequential(
                        nn.Linear(128, size),
                        nn.Softmax(dim=1),
                    )  # TODO: refine decoder structure, e.g. add more layers
                    for param_name, size in classification_params.items()
                }
                if classification_params
                else {}
        )
        self.regression_tail = nn.ModuleDict(
            {
                param_name: nn.Linear(128, size)  # TODO: refine decoder structure, e.g. add more layers
                for param_name, size in regression_params.items()
            }
            if regression_params
            else {}
        )

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        classification_outputs = {
            param_name: tail(x) for param_name, tail in self.classification_tails.items()
        } if self.classification_tails else {}
        regression_output = {
            param_name: tail(x) for param_name, tail in self.regression_tail.items()
        } if self.regression_tail else {}
        return classification_outputs, regression_output

In [10]:
class EncoderDecoderModel(nn.Module):
    def __init__(self, encoder, decoders):
        super(EncoderDecoderModel, self).__init__()
        self.encoder = encoder
        self.decoders = decoders

    def forward(self, x):
        x = self.encoder(x)
        batch_size = x.size(0)  # Get the batch size
        x = x.view(batch_size, -1)  # Flatten the feature tensor, considering the batch size
        decoder_outputs = {decoder_name: decoder(x) for decoder_name, decoder in self.decoders.items()}
        return decoder_outputs  # note that the multi-tail decoder returns a list of outputs

In [11]:
class DAGDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_name: str, datasets_folder: str="./datasets", transform=None):
        self.dataset_name = dataset_name
        self.datasets_folder = datasets_folder
        self.dataset_path = os.path.join(self.datasets_folder, self.dataset_name)
        self.images_folder = os.path.join(self.dataset_path, "images")
        self.params_folder = os.path.join(self.dataset_path, "params")
        self.ranges_file_path = os.path.join(self.dataset_path, "ranges.yml")
        self.ranges = None
        self.decoders = None
        self.transform = transforms.Compose(
            [transforms.Resize((512, 512)), transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
            ) if transform is None else transform
        self.data = self.load_data()

    def load_data(self):
        # Load the ranges from the YAML file
        with open(self.ranges_file_path, 'r') as file:
            self.ranges = yaml.safe_load(file)
        # Load the decoders from the YAML file
        with open(os.path.join(self.dataset_path, "decoders.yml"), 'r') as file:
            self.decoders = yaml.safe_load(file)
        # read images and parameters
        data = []
        for image_name in os.listdir(self.images_folder):
            image_path = os.path.join(self.images_folder, image_name)
            param_path = os.path.join(self.params_folder, os.path.splitext(image_name)[0] + ".yml")
            with open(param_path, 'r') as file:
                param = yaml.safe_load(file)
            # normalize
            param = self.preprocess(param)
            param = self.format_target_to_decoders(param)
            data.append((image_path, param))
        return data

    def format_target_to_decoders(self, target):
        formatted_target = {}
        for decoder_name, decoder_params in self.decoders.items():
            formatted_target[decoder_name] = {
                "classification_targets": {},
                "regression_target": {}
            }
            for param_name in decoder_params:
                param_type = self.ranges[param_name]['type']
                if param_type == 'float' or param_type == 'int' or param_type == 'vector':
                    formatted_target[decoder_name]['regression_target'][param_name] = target[param_name]
                elif param_type == 'states' or param_type == 'bool':
                    formatted_target[decoder_name]['classification_targets'][param_name] = target[param_name]
        return formatted_target

    def preprocess(self, param):
        processed_param = {}
        # for float and vector: normalize with min max
        # for states, bool: convert to one hot
        # for ints: treat as float, but round back to int when saving as param
        for param_name, param_spec in self.ranges.items():
            if param_spec['type'] == 'float' or param_spec['type'] == 'int' or param_spec['type'] == 'vector':
                processed_param[param_name] = self.normalize(param[param_name], param_spec)
            elif param_spec['type'] == 'states' or param_spec['type'] == 'bool':
                processed_param[param_name] = self.one_hot(param[param_name], param_spec)
            else:
                raise ValueError(f"Unsupported parameter type: {param_spec['type']}")
        return processed_param

    def normalize(self, value, param_spec):
        if param_spec['type'] == 'float' or param_spec['type'] == 'int':
            return (value - param_spec['min']) / (param_spec['max'] - param_spec['min'])
        elif param_spec['type'] == 'vector':
            return [(value[i] - param_spec[f'{dim}min']) / (param_spec[f'{dim}max'] - param_spec[f'{dim}min']) for i, dim in enumerate(['x', 'y', 'z'])]
        else:
            raise ValueError(f"Unsupported parameter type: {param_spec['type']}")

    def one_hot(self, value, param_spec):
        if param_spec['type'] == 'states':
            index = param_spec['values'].index(value)
            return [1 if i == index else 0 for i in range(len(param_spec['values']))]
        elif param_spec['type'] == 'bool':
            # make bools onehot too to make it consistent
            return [1, 0] if value else [0, 1]
        else:
            raise ValueError(f"Unsupported parameter type: {param_spec['type']}")


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample, target = self.data[idx]
        sample = Image.open(sample).convert('L')

        if self.transform:
            sample = self.transform(sample)

        # convert target's values to tensor
        for decoder_name, decoder_outputs in target.items():
            # check if is tensor
            for classification_target in decoder_outputs['classification_targets']:
                if not torch.is_tensor(decoder_outputs['classification_targets'][classification_target]):
                    target[decoder_name]['classification_targets'][classification_target] = torch.tensor(decoder_outputs['classification_targets'][classification_target], dtype=torch.float32)
            for regression_target in decoder_outputs['regression_target']:
                if not torch.is_tensor(decoder_outputs['regression_target'][regression_target]):
                    target[decoder_name]['regression_target'][regression_target] = torch.tensor(decoder_outputs['regression_target'][regression_target], dtype=torch.float32)

        return sample, target

In [12]:
dataset = DAGDataset("test_dataset")

In [13]:
dataset.__getitem__(0)

(tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]),
 {'Building Mass Decoder': {'classification_targets': {'Bm Base Shape': tensor([0., 1.])},
   'regression_target': {'Bm Size': tensor([0.5536, 0.4045, 0.3762])}},
  'Facade Decoder': {'classification_targets': {},
   'regression_target': {'Num Floors': tensor(0.2500),
    'Num Windows Each Side': tensor(0.)}},
  'Floor Ledge Decoder': {'classification_targets': {'Has Floor Ledge': tensor([0., 1.])},
   'regression_target': {'Floor Ledge Size X': tensor(0.1306),
    'Floor Ledge Size Z': tensor(0.5549),
    'Floor Ledge Extrusion X': tensor(0.8421),
    'Floor Ledge Extrusion Z': tensor(0.4717)}},
  'Roof Decoder': {'classification_targets': {'Rf Base Shape': tensor([1., 0., 0.])},
   'regression_target': {'Rf Size': tens

In [14]:
# split into train val and test
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Create DataLoader
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [15]:
encoder = Encoder()

In [16]:
# Load the ranges from the YAML file
with open('./datasets/test_dataset/ranges.yml', 'r') as file:
    ranges = yaml.safe_load(file)

# Create a mapping between parameter names and output sizes
parameter_output_mapping = {}
for decoder_name, param_specs in ranges.items():
    if param_specs['type'] == 'float':
        parameter_output_mapping[decoder_name] = 1  # 1 for scalar
    elif param_specs['type'] == 'int':
        parameter_output_mapping[decoder_name] = 1  # 1 for scalar
    elif param_specs['type'] == 'vector':
        parameter_output_mapping[decoder_name] = 3  # 3 for x, y, z
    elif param_specs['type'] == 'states':
        parameter_output_mapping[decoder_name] = len(param_specs['values'])
    elif param_specs['type'] == 'bool':
        parameter_output_mapping[decoder_name] = 2  # 2 for binary encoding

In [17]:
# Load the decoders' params from the YAML file
with open('./datasets/test_dataset/decoders.yml', 'r') as file:
    decoders_params = yaml.safe_load(file)

decoders = nn.ModuleDict()
# initialize decoders with corresponding output tails
for decoder_name, param_names in decoders_params.items():
    classification_tails = {}
    regression_tails = {}
    for param_name in param_names:
        spec = ranges[param_name]
        # if type is bool or states, add to classification tails
        # if type is float, int or vector, add to regression tails
        if spec['type'] == 'bool' or spec['type'] == 'states':
            classification_tails[param_name] = parameter_output_mapping[param_name]
        else:
            regression_tails[param_name] = parameter_output_mapping[param_name]
    # add decoder to model
    decoders[decoder_name] = ParamAwareMultiTailDecoder(1024, classification_tails, regression_tails)

# decoders = nn.ModuleDict({
#     param_name: Decoder(4096, output_size)
#     for param_name, output_size in parameter_output_mapping.items()
# })

In [18]:
model = EncoderDecoderModel(encoder, decoders)

In [19]:
model

EncoderDecoderModel(
  (encoder): Encoder(
    (features): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (9): Flatten(start_dim=1, end_dim=-1)
      (10): Linear(in_features=131072, out_features=1024, bias=True)
    )
  )
  (decoders): ModuleDict(
    (Building Mass Decoder): ParamAwareMultiTailDecoder(
      (fc1): Linear(in_features=1024, out_features=256, bias=True)
      (relu1): ReLU()
      (fc2): Linear(in_features=256, out_features=128, b

In [20]:
# !pip install torchinfo
# from torchinfo import summary

# batch_size = 8
# summary(model, input_size=(batch_size, 1, 512, 512))

# Basically my model is too unique

In [21]:
# Load the switches from the YAML file
with open('./datasets/test_dataset/switches.yml', 'r') as file:
    switches = yaml.safe_load(file)

In [25]:
# Define loss function and optimizer
# for regression, use MSELoss, for classification, use CrossEntropyLoss
class EncDecsLoss(nn.Module):
    def __init__(self, decoders, switches_mapping: dict):
        super(EncDecsLoss, self).__init__()
        self.decoders = decoders
        self.switches_mapping = switches_mapping

    def forward(self, outputs, targets):
        loss = 0.0
        for decoder_name, decoder_output in outputs.items():
            loss += self.decoder_loss(decoder_output, targets[decoder_name])
        return loss

    def classification_loss(self, output, target):
        return nn.CrossEntropyLoss()(output, target)

    def regression_loss(self, output, target):
        return nn.MSELoss()(output, target)

    # def find_param_name_in(target, param_name):
    #     for decoder_name, decoder_outputs in target.items():
    #         for i, classification_target in enumerate(decoder_outputs['classification_targets']):
    #             if param_name ==
    #     return None

    def decoder_loss(self, decoder_output, target):
        classification_outputs = decoder_output[0]  # note that model outputs a tuple of list instead of dict of list
        regression_output = decoder_output[1]
        total_classification_loss = 0.0
        # if classification_outputs:
        for param_name, pred in classification_outputs.items():
            total_classification_loss += self.classification_loss(pred, target["classification_targets"][param_name])
            # TODO: should we add early termination for "Has" labels?
        # if regression_output:
        total_regression_loss = 0.0
        for param_name, pred in regression_output.items():
            regression_loss = self.regression_loss(pred, target["regression_target"][param_name])
            # use gt's 0 1 label to switch off the loss if needed
            switch_param_name = self.switches_mapping["Reversed Mapping"].get(param_name)
            if switch_param_name:
                switch_target = target["classification_targets"][switch_param_name]
                switch_index = torch.argmax(switch_target, dim=1)
                # make regression_loss same shape as switch_index
                regression_loss = torch.stack([regression_loss] * switch_index.size(0))
                regression_loss *= switch_index
                # average again
                regression_loss = torch.mean(regression_loss)
            total_regression_loss += regression_loss
        averaged_classification_loss = total_classification_loss / len(classification_outputs) if len(classification_outputs) > 0 else 0
        averaged_regression_loss = total_regression_loss / len(regression_output) if len(regression_output) > 0 else 0
        loss = averaged_classification_loss + averaged_regression_loss
        return loss

criterion = EncDecsLoss(decoders, switches)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [26]:
# Training loop with train and val
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, targets = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 200 == 199:  # print every 200 mini-batches
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 2000:.3f}")
            running_loss = 0.0
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(val_loader, 0):
            inputs, targets = data
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()
        print(f"Validation loss: {val_loss / len(val_loader)}")

print("Finished Training")

# Save your trained model if needed
torch.save(model.state_dict(), "encDecModel.pth")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Validation loss: 5.056028366088867
Validation loss: 4.685829162597656
Validation loss: 4.671940803527832
Validation loss: 4.270366191864014
Validation loss: 4.4407501220703125
Validation loss: 4.388157367706299
Validation loss: 4.363568305969238
Validation loss: 4.229415416717529
Validation loss: 4.260926246643066
Validation loss: 4.180829048156738
Finished Training


In [27]:
# Test the model
model.eval()
test_loss = 0.0
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        inputs, targets = data
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
    print(f"Test loss: {test_loss / len(test_loader)}")
print("Finished Testing")

Test loss: 4.420920372009277
Finished Testing


In [28]:
outputs

{'Building Mass Decoder': ({'Bm Base Shape': tensor([[0.1183, 0.8817],
           [0.1068, 0.8932],
           [0.1096, 0.8904],
           [0.1339, 0.8661],
           [0.1114, 0.8886],
           [0.1100, 0.8900],
           [0.1164, 0.8836],
           [0.1192, 0.8808],
           [0.1115, 0.8885],
           [0.1036, 0.8964]])},
  {'Bm Size': tensor([[0.6022, 0.5549, 0.2318],
           [0.6433, 0.5849, 0.2420],
           [0.6312, 0.5758, 0.2398],
           [0.5512, 0.5171, 0.2211],
           [0.6249, 0.5714, 0.2379],
           [0.6302, 0.5751, 0.2393],
           [0.6083, 0.5592, 0.2337],
           [0.5987, 0.5520, 0.2316],
           [0.6254, 0.5718, 0.2381],
           [0.6553, 0.5939, 0.2445]])}),
 'Facade Decoder': ({},
  {'Num Floors': tensor([[0.3763],
           [0.3954],
           [0.3907],
           [0.3506],
           [0.3874],
           [0.3897],
           [0.3788],
           [0.3739],
           [0.3877],
           [0.4018]]),
   'Num Windows Each Side': te

In [24]:
targets

{'Building Mass Decoder': {'classification_targets': {'Bm Base Shape': tensor([[0., 1.],
           [0., 1.],
           [0., 1.],
           [0., 1.],
           [0., 1.],
           [1., 0.],
           [0., 1.],
           [1., 0.],
           [0., 1.],
           [0., 1.],
           [1., 0.],
           [1., 0.],
           [0., 1.],
           [1., 0.],
           [1., 0.],
           [1., 0.],
           [0., 1.],
           [0., 1.],
           [1., 0.],
           [0., 1.],
           [0., 1.],
           [0., 1.],
           [0., 1.],
           [1., 0.],
           [0., 1.],
           [1., 0.],
           [0., 1.],
           [0., 1.],
           [0., 1.],
           [0., 1.],
           [0., 1.],
           [0., 1.]])},
  'regression_target': {'Bm Size': tensor([[0.3605, 0.4609, 0.1077],
           [0.5902, 0.4303, 0.3231],
           [0.5866, 0.4268, 0.3613],
           [0.5151, 0.4979, 0.3843],
           [0.8842, 0.9664, 0.5252],
           [0.0413, 0.2278, 0.2861],
   